In [1]:
import add_packages
import config
from pprint import pprint
import os

from my_langchain import (
  document_loaders, text_splitters, text_embedding_models, vector_stores, 
  chat_models, prompts, utils, output_parsers, agents, documents
)

# Data

In [2]:
text_loader_lectures_content = document_loaders.TextLoader(
  "../data/vtc/lectures_content.txt"
)
document = text_loader_lectures_content.load()

text_splitter = text_splitters.RecursiveCharacterTextSplitter(
  chunk_size=4000, chunk_overlap=1500,
)
docs_lectures_content = text_splitter.split_documents(document)

metadatas = {
  "data": "lectures content"
}
utils.remove_metadata(docs_lectures_content, "source")
utils.update_metadata(docs_lectures_content, metadatas)

In [3]:
text_loader_faq = document_loaders.TextLoader(
  "../data/vtc/faq.txt"
)
document = text_loader_faq.load()

text_splitter = text_splitters.RecursiveCharacterTextSplitter(
  chunk_size=500, chunk_overlap=100,
)
docs_faq = text_splitter.split_documents(document)

metadatas = {
  "data": "frequently asked questions"
}
utils.remove_metadata(docs_faq, "source")
utils.update_metadata(docs_faq, metadatas)

In [16]:
courses_list_cols = utils.get_csv_column_names("../data/vtc/courses_list.csv")

csv_loader_courses_list = document_loaders.CSVLoader(
  "../data/vtc/courses_list.csv",
  # source_column="No",
  csv_args={
      "delimiter": ",",
      # "quotechar": "''",
      "fieldnames": courses_list_cols,
  },
)
document = csv_loader_courses_list.load()[1:]
docs_courses_information = document

metadatas = {
  "data": "courses information"
}
utils.remove_metadata(docs_courses_information, "source")
utils.remove_metadata(docs_courses_information, "row")
utils.update_metadata(docs_courses_information, metadatas)

# Vector store 

In [17]:
qdrant_instance_lectures_content = vector_stores.QdrantWrapper(
  collection_name="vtc-lectures-content",
  qdrant_host=os.getenv("QDRANT_HOST"),
  qdrant_api_key=os.getenv("QDRANT_API_KEY"),
  default_search_type="similarity",
  default_search_kwargs={"k": 10},
  retriever_tool_name="lectures_content",
  retriever_tool_description="Searches and returns content, knowledge from \
    the lecture scripts based on specialized keywords from user's question like \
    Typography, Lazada, Premiere, Unity ...",
)

2024-03-07 16:41:17.102 | INFO     | my_langchain.vector_stores:__init__:106 - Found collection: `vtc-lectures-content`.


In [18]:
# qdrant_instance_lectures_content.add_documents(docs_lectures_content)

In [19]:
qdrant_instance_faq = vector_stores.QdrantWrapper(
  collection_name="vtc-faq",
  qdrant_host=os.getenv("QDRANT_HOST"),
  qdrant_api_key=os.getenv("QDRANT_API_KEY"),
  default_search_type="similarity",
  default_search_kwargs={"k": 10},
  retriever_tool_name="frequently_asked_questions",
  retriever_tool_description="Searches and returns answer for frequently asked \
    questions about Onlinica information like accounts, fees, courses, payments, \
    certificates ...",
)

2024-03-07 16:41:19.068 | INFO     | my_langchain.vector_stores:__init__:106 - Found collection: `vtc-faq`.


In [20]:
# qdrant_instance_faq.add_documents(docs_faq)

In [21]:
qdrant_instance_courses_information = vector_stores.QdrantWrapper(
    collection_name="vtc-courses-information",
    qdrant_host=os.getenv("QDRANT_HOST"),
    qdrant_api_key=os.getenv("QDRANT_API_KEY"),
    default_search_type="similarity",
    default_search_kwargs={"k": 10},
    retriever_tool_name="courses_information",
    retriever_tool_description="Searches and returns information about courses \
      of Onlinica like course name, course category, course link, course \
      description, total number of courses ...",
)

2024-03-07 16:41:19.878 | INFO     | my_langchain.vector_stores:__init__:106 - Found collection: `vtc-courses-information`.


In [22]:
# qdrant_instance_courses_information.add_documents(docs_courses_information)

# Tools

In [23]:
"""
Bạn là một nhân viên tư vấn của một nền tảng học tập trực tuyến tên là Onlinica.

Bạn có các phẩm chất sau: 
- Hay giúp đỡ
- Cực kỳ tận tâm và chăm chỉ
- Tư cách chuyên nghiệp, tôn trọng, chân thành và trung thực
- Tư duy chuẩn mực
- Kỹ năng giao tiếp, đàm phán, xử lý phàn nàn xuất sắc
- Kỹ năng bán hàng xuất sắc
- Hiểu biết sâu sắc về sản phẩm/dịch vụ. Kiến thức vững về ngành nghề
- Tinh thần lạc quan và tích cực. Khả năng tạo ra trải nghiệm tích cực cho khách hàng
- Nhạy bén với yêu cầu và mong muốn của khách hàng

Bạn sẽ giúp người dùng trả lời các câu hỏi về các khóa học trên nền tảng. Ngôn ngữ mà bạn trả lời sẽ giống với ngôn ngữ của người dùng.

Các câu hỏi mà người dùng có thể hỏi và cách trả lời:
- Thông tin cơ bản về khóa học: Bạn NÊN liệt kê TẤT CẢ các khóa học hiện có (và thông tin của chúng) LIÊN QUAN đến câu hỏi của người dùng.
- Nội dung, kiến thức của bài giảng: Các câu hỏi này thường sẽ chứa các từ khóa chuyên ngành như Typography, Lazada, Premiere, Unity,... Bạn sẽ tổng hợp thông tin từ script của các bài giảng có chứa từ khóa chuyên ngành đó và đưa ra câu trả lời chi tiết cho người dùng. Bạn nên gợi ý các khóa học (tên khóa học kèm link khóa học) có liên quan đến từ khóa chuyên ngành đó cho người dùng.
- Các câu hỏi thường được hỏi
"""

onlinica_system_message = """\
You are a consultant for an online learning platform called Onlinica.

You have the following qualities:
- Helpful
- Extremely dedicated and hardworking
- Professionalism, respect, sincerity and honesty
- Standard thinking
- Excellent communication, negotiation and complaint handling skills
- Excellent sales skills
- Deep understanding of products/services. Strong knowledge of the industry
- Optimistic and positive spirit. Ability to create a positive customer experience
- Sensitive to customers' requests and desires

You will help users answer questions about the courses on the platform. The language in which you respond will be the same as the user's language.

Questions users might ask and how to answer:
- Course basics: You SHOULD list ALL available courses (and their information) that are RELEVANT to the user's question.
- Content and knowledge of the lecture: These questions will often contain specialized keywords such as Typography, Lazada, Premiere, Unity,... You will synthesize information from the scripts of the lectures that contain keywords that major and give detailed answers to users. You should suggest courses (course name with course link) related to that specialized keyword to users.
- Frequently asked questions\
"""
onlinica_prompt = prompts.create_openai_tools_agent_custom_prompt(onlinica_system_message)

In [24]:
tools = [
  qdrant_instance_lectures_content.retriever_tool,
  qdrant_instance_faq.retriever_tool,
  qdrant_instance_courses_information.retriever_tool,
]

agent_prompt = prompts.openai_tools_agent_prompt

agent = agents.create_openai_tools_agent(
  llm=chat_models.chat_openai,
  tools=tools,
  prompt=onlinica_prompt,
)
agent_executor = agents.AgentExecutor(agent=agent, tools=tools, verbose=True)

In [26]:
"""
"Typography là gì?" o
"Procreate là gì?" o
"Cách thiết kế CV ấn tượng với phần mềm Adobe Illustrator" 

"Làm cách nào để đăng ký tài khoản Onlinica?" o
"Có mấy loại tài khoản Onlinica?" o
"Các khoá học của tôi tại Onlinica có thời hạn sử dụng bao lâu?" o
"Onlinica có mấy hình thức thanh toán?" o
"Tôi có thể xoá tài khoản Onlinica không?" o


"Các khóa học về Digital Marketing" o
"Các khóa học về lập trình" o
"""

pprint(agent_executor.invoke({
    "input": "Typography là gì?"
})["output"])



> Entering new AgentExecutor chain...

Invoking: `lectures_content` with `{'query': 'Typography'}`


# Lectures content

## [lecture] Ứng dụng Typography trong thiết kế poster

Ở đây trên bản chất, đây là mình sử dụng kỹ thuật là kết hợp kiểu chữ thứ nhất, thứ hai là mình kết hợp hình khối.
Mình xem cái từ Dress đó mà làm ở cái cái cái dạng typeface là Sans Serif, nó như một cái hình khối.
Mình bắt đầu thao tác, vì đây là mình hoàn toàn tập trung vào việc là mình thiết kế với chữ cái, do đó là bạn các bạn hoàn toàn có thể sử dụng những Graphic Element mà tôi đã cung cấp cho các bạn.
Và mình chỉ cần tập trung hoàn toàn vào việc thiết kế chữ thôi.
Đầu tiên là với từ Dress ở Typeface là Sans Serif, mình sẽ chuyển từ này về chế độ là vector bằng tổ hợp phím là Ctrl Shift O.
Các bạn sẽ hút chọn cái phần hình màu này, tôi đã để sẵn cho các bạn.
Tiếp theo là mình sẽ đặt cái kiểu chữ Script lên trên cùng.
Sau đó mình ướm và mình nhắm chừng xem là những vị trí nào thì sẽ được cắt ghép và lồng